IMPORTACIÓN DE LIBRERIAS

In [1]:
from google.cloud import bigquery
from cmath import nan
import pandas as pd
import pytz
import numpy as np
import itertools
from tqdm.notebook import tqdm_notebook

# credenciales airflow
#_google_key = "/home/airflow/airflow/dags/servacc_bigquery.json"
_google_key = "../Credenciales/servacc_bigquery.json"

SE CREA DF USUARIOS_PROGRAMAS PARA COMBINAR PROGRAMAS Y USUARIOS

In [2]:
query_usuarios_programas = """
WITH programas AS (
  SELECT
    dicc.programa, dicc.soporte, content_id,
    datetime(fecha, dicc.inicio_oficial) AS hora_inicio,
    datetime(fecha, dicc.fin_oficial) AS hora_fin,
  FROM UNNEST(GENERATE_DATE_ARRAY("2022-03-15",CURRENT_DATE(),INTERVAL 1 day)) AS fecha
  CROSS JOIN `conexion-datos-rdf.diccionarios.dicc_programas` AS dicc
  WHERE EXTRACT(dayofweek FROM fecha) IN (SELECT dias FROM UNNEST(dicc.dias) dias)
    AND dicc.activo
  ORDER BY hora_inicio DESC
)
  SELECT
    programas.soporte,
    programas.programa,
    consumo.content_id,
    consumo.user_id,
    programas.hora_inicio AS periodo,
  FROM `conexion-datos-rdf.consumo.consumo_detalle` AS consumo
  JOIN `webhooks.usuarios_unicos` as users
    on consumo.user_id = users.platform_id
  JOIN programas
  ON datetime(consumo.end_date, "America/Santiago") >= programas.hora_inicio 
    AND datetime(consumo.start_date,"America/Santiago") < programas.hora_fin
    AND consumo.content_id = programas.content_id
  WHERE consumo.start_date >= "2022-03-13"
  AND user_id IS NOT NULL
    """

client = bigquery.Client.from_service_account_json(_google_key)
usuarios_programas = client.query(query_usuarios_programas).result(
).to_dataframe(create_bqstorage_client=False)

""" Se crea columna día que extrae date de la columna periodo, con el própósito de 
    usarla como columna de unión al momento de crear el df_super, que es la unión 
    de usuarios_programas consigo misma.
"""
usuarios_programas['dia'] = usuarios_programas['periodo'].dt.date


SE CREA DF SUPER QUE COMBINA DE PROGRAMA_REFERENCIA CON TODOS LOS PROGRAMAS

In [3]:
df_super = pd.merge(usuarios_programas, usuarios_programas, how="left", on=["soporte", "content_id", "user_id", "dia"])
df_super = df_super.rename(columns={"programa_x": "programa", "periodo_x": "periodo",
                           "programa_y": "programa_referencia", "periodo_y": "periodo_referencia"}).copy()
df_super = df_super[["soporte", "content_id", "programa", "periodo", "programa_referencia", "periodo_referencia", "user_id"]]

SE CREA DF_SUPER_COUNT PARA CONTAR USUARIOS POR PROGRAMA

In [4]:
df_super_count = df_super.copy()
df_super_count = df_super_count.groupby(by=["soporte", "content_id", "programa", "periodo", "programa_referencia", "periodo_referencia"])[
            "user_id"].nunique().to_frame(name='usuarios').reset_index()

SE CREA DF_PROGRAMAS PARA LOGRAR DF_COMB QUE ES LA GRILLA 

In [5]:
query_programas = """
SELECT
    dicc.programa, 
    dicc.soporte, 
    content_id,
    datetime(fecha, dicc.inicio_oficial) AS hora_inicio,
    datetime(fecha, dicc.fin_oficial) AS hora_fin,
  FROM UNNEST(GENERATE_DATE_ARRAY("2022-03-15",CURRENT_DATE(),INTERVAL 1 day)) AS fecha
  CROSS JOIN `conexion-datos-rdf.diccionarios.dicc_programas` AS dicc
  WHERE EXTRACT(dayofweek FROM fecha) IN (SELECT dias FROM UNNEST(dicc.dias) dias)
    AND dicc.activo
"""
df_programas = client.query(query_programas).result(
).to_dataframe(create_bqstorage_client=False)
df_programas['dia'] = df_programas['hora_inicio'].dt.date

SE CREA DF_COMB CON TODOS LOS PROGRAMAS COMBINADOS CON PROGRAMA_REFERENCIA

In [6]:
# esta es mi grilla, es decir, me trae el cruce de todos los programas independiente de si consumieron usuarios o no
df_comb = pd.merge(df_programas, df_programas, how="inner", on=["content_id", "dia"])
df_comb = df_comb.drop(["hora_fin_x", "hora_fin_y", "soporte_y", "dia"], axis=1).copy()
df_comb = df_comb.rename(columns={"programa_x": "programa", "soporte_x": "soporte", "hora_inicio_x": "periodo",
                                  "programa_y": "programa_referencia", "hora_inicio_y": "periodo_referencia"})
df_comb['hora'] = df_comb['periodo'].dt.time
df_comb['hora_referencia'] = df_comb['periodo_referencia'].dt.time
df_comb = df_comb[["content_id", "soporte", "programa", "periodo", "hora", "programa_referencia",
                  "periodo_referencia", "hora_referencia"]]

SE CREA DF_USUARIOS 

In [7]:
query_usuarios = """
    Select 
    users.platform_id as user_id,
    users.gender,
    tramo.nombre,
    tramo.desc_tramo
    from `conexion-datos-rdf.webhooks.usuarios_unicos` as users
    left join `conexion-datos-rdf.diccionarios.dicc_tramo_etario` as tramo
    on ABS(CAST(TRUNC(DATE_DIFF(CURRENT_DATE(), DATE(birthday), month) / 12) AS INTEGER )) 
    between tramo.edad_inf and tramo.edad_sup
"""
df_users = client.query(query_usuarios).result().to_dataframe(create_bqstorage_client=False)
df_users['gender'] = df_users['gender'].replace(['FEMALE', 'MALE', 'OTHER'], ['femenino', 'masculino', 'otro género'])

SE CREAN DF_USUARIOS POR SEGMENTOS GÉNERO, RADIAL, DIGITAL Y COMBINADOS

In [8]:
df_users_genero = df_users[['user_id', 'gender']]
df_users_femenino = df_users_genero[df_users_genero['gender'] == 'femenino']
df_users_masculino = df_users_genero[df_users_genero['gender'] == 'masculino']
df_users_otro = df_users_genero[df_users_genero['gender'] == 'otro género']

df_users_tramo = df_users[['user_id', 'nombre', 'desc_tramo']]
df_users_radial = df_users_tramo[df_users_tramo['nombre'] == 'Radial']
df_users_r_otros = df_users_radial[df_users_radial['desc_tramo'] == 'otros edad']
df_users_r_15a19 = df_users_radial[df_users_radial['desc_tramo'] == '15 - 19']
df_users_r_20a24 = df_users_radial[df_users_radial['desc_tramo'] == '20 - 24']
df_users_r_25a34 = df_users_radial[df_users_radial['desc_tramo'] == '25 - 34']
df_users_r_35a44 = df_users_radial[df_users_radial['desc_tramo'] == '35 - 44']
df_users_r_45a59 = df_users_radial[df_users_radial['desc_tramo'] == '45 - 59']
df_users_r_60 = df_users_radial[df_users_radial['desc_tramo'] == '60 o más']

df_users_digital = df_users_tramo[df_users_tramo['nombre'] == 'Digital']
df_users_d_otros = df_users_digital[df_users_digital['desc_tramo'] == 'otros edad']
df_users_d_18a24 = df_users_digital[df_users_digital['desc_tramo'] == '18 - 24']
df_users_d_25a34 = df_users_digital[df_users_digital['desc_tramo'] == '25 - 34']
df_users_d_35a44 = df_users_digital[df_users_digital['desc_tramo'] == '35 - 44']
df_users_d_45a54 = df_users_digital[df_users_digital['desc_tramo'] == '45 - 54']
df_users_d_55a64 = df_users_digital[df_users_digital['desc_tramo'] == '55 - 64']
df_users_d_65 = df_users_digital[df_users_digital['desc_tramo'] == '65 o más']

df_users_femenino_radial = df_users[(df_users.gender == 'femenino') & (df_users.nombre == 'Radial')]
df_users_femenino_digital = df_users[(df_users.gender == 'femenino') & (df_users.nombre == 'Digital')]
df_users_masculino_radial = df_users[(df_users.gender == 'masculino') & (df_users.nombre == 'Radial')]
df_users_masculino_digital = df_users[(df_users.gender == 'masculino') & (df_users.nombre == 'Digital')]
df_users_otrogen_radial = df_users[(df_users.gender == 'otro género') & (df_users.nombre == 'Radial')]
df_users_otrogen_digital = df_users[(df_users.gender == 'otro género') & (df_users.nombre == 'Digital')]

SE CREAN LISTAS DE USUARIOS SEGÚN SEGMENTO PARA USAR EN LAS FUNCIONES

In [9]:
df_list_genero = [df_users_femenino, df_users_masculino, df_users_otro]
df_list_tramo = [df_users_r_otros, df_users_r_15a19, df_users_r_20a24,
                 df_users_r_25a34, df_users_r_35a44, df_users_r_45a59, df_users_r_60]
df_list_digital = [df_users_d_otros, df_users_d_18a24, df_users_d_25a34,
                   df_users_d_35a44, df_users_d_45a54, df_users_d_55a64, df_users_d_65]
df_list_combinados = [df_users_femenino_radial, df_users_femenino_digital, df_users_masculino_radial, 
                        df_users_masculino_digital, df_users_otrogen_radial, df_users_otrogen_digital]

SE CREA FUNCIÓN para obtener segmentos Género

In [10]:
def gen_df_genero(df_super: pd.DataFrame, df_super_count: pd.DataFrame, df_comb: pd.DataFrame, list_df_segmento: list):

    df_super_genero = pd.DataFrame()

    for df_segmento in tqdm_notebook(list_df_segmento):

        # generamos df_super
        df_super_seg = pd.merge(df_super, df_segmento,
                                how="inner",  on="user_id")
        # generamos df_super_count
        df_super_count_seg = df_super_seg.groupby(by=["soporte", "content_id", "programa", "periodo", "programa_referencia",
                                                  "periodo_referencia", "gender"])["user_id"].nunique().to_frame(name='usuarios').reset_index()
        # #cruzamos df_comb con df_super_femenino_count para obtener total de usuarios que consumen programa de referencia con programa

        df_superpos_1 = pd.merge(df_comb, df_super_count_seg, how="inner", on=["content_id", "periodo", "programa", "periodo_referencia", "programa_referencia"])
        del df_superpos_1['soporte_y']
        df_superpos_1['dia'] = df_superpos_1['periodo'].dt.date
        df_superpos_1 = df_superpos_1.rename(columns={"soporte_x": "soporte", "gender": "segmento"}).copy()
        df_superpos_1 = df_superpos_1[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "segmento"]]
        df_superpos_1.usuarios = df_superpos_1.usuarios.fillna(0)

        df_superpos_2 = df_superpos_1.copy()
        df_superpos_2['key_periodo'] = df_superpos_2['periodo_referencia']
        df_superpos_2['key_programa'] = df_superpos_2['programa_referencia']

        df_super_count = df_super_count.rename(columns={'periodo': 'key_periodo', 'programa': 'key_programa'})

        df_superpos = pd.merge(df_superpos_2, df_super_count, how="left", on=["key_periodo", "periodo_referencia", "key_programa", "programa_referencia", "content_id"])
        df_superpos = df_superpos.drop(["key_periodo", "key_programa", "soporte_y"], axis=1).copy()
        df_superpos = df_superpos.rename(columns={"soporte_x": "soporte", "usuarios_x": "usuarios", "usuarios_y": "total"})
        df_superpos['tipo_tramo'] = "Género"
        df_superpos = df_superpos[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "total", "tipo_tramo", "segmento"]]

        df_superpos.total = df_superpos.total.fillna(0)
        df_superpos["usuarios"] = df_superpos['usuarios'].astype('int64')
        df_superpos["total"] = df_superpos['total'].astype('int64')
        df_superpos = df_superpos[df_superpos['segmento'].notna()]

        df_super_genero = pd.concat(objs=[df_super_genero, df_superpos], axis=0)

    return df_super_genero

SE CREA FUNCIÓN para obtener segmento EDAD Radial

In [11]:
def gen_df_tramos(df_super: pd.DataFrame, df_super_count: pd.DataFrame, df_comb: pd.DataFrame, list_df_segmento: list):

    df_super_tramo = pd.DataFrame()

    for df_segmento in tqdm_notebook(list_df_segmento):

        df_super_r = pd.merge(df_super, df_segmento, how="inner", on="user_id")

        df_super_r_count = df_super_r.groupby(by=["soporte", "content_id", "programa", "periodo", "programa_referencia",
                                                  "periodo_referencia", "nombre", "desc_tramo"])["user_id"].nunique().to_frame(name='usuarios').reset_index()

        df_superpos_r_1 = pd.merge(df_comb, df_super_r_count, how="inner", on=["content_id", "periodo", "programa", "periodo_referencia", "programa_referencia"])
        del df_superpos_r_1['soporte_y']
        df_superpos_r_1['dia'] = df_superpos_r_1['periodo'].dt.date
        df_superpos_r_1 = df_superpos_r_1.rename(columns={"soporte_x": "soporte", "nombre": "tipo_tramo", "desc_tramo": "segmento"}).copy()
        df_superpos_r_1 = df_superpos_r_1[["dia", "content_id", "soporte", "programa", "periodo", "hora","programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "tipo_tramo", "segmento"]]
        df_superpos_r_1.usuarios = df_superpos_r_1.usuarios.fillna(0)

        df_superpos_r_2 = df_superpos_r_1.copy()
        df_superpos_r_2['key_periodo'] = df_superpos_r_2['periodo_referencia']
        df_superpos_r_2['key_programa'] = df_superpos_r_2['programa_referencia']
        
        df_super_count = df_super_count.rename(columns={'periodo': 'key_periodo', 'programa': 'key_programa'})

        df_superpos_r = pd.merge(df_superpos_r_2, df_super_count, how="left", on=["key_periodo", "periodo_referencia", "key_programa", "programa_referencia", "content_id"])
        df_superpos_r = df_superpos_r.drop(["key_periodo", "key_programa", "soporte_y"], axis=1).copy()
        df_superpos_r = df_superpos_r.rename(columns={"soporte_x": "soporte", "usuarios_x": "usuarios", "usuarios_y": "total"})
        df_superpos_r = df_superpos_r[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "total", "tipo_tramo", "segmento"]]

        df_superpos_r.total = df_superpos_r.total.fillna(0)
        df_superpos_r["usuarios"] = df_superpos_r['usuarios'].astype('int64')
        df_superpos_r["total"] = df_superpos_r['total'].astype('int64')
        df_superpos_r = df_superpos_r[df_superpos_r['segmento'].notna()]

        df_super_tramo = pd.concat(objs=[df_super_tramo, df_superpos_r], axis=0)

    return df_super_tramo

SE CREA FUNCIÓN para pbtener segmento EDAD Digital

In [12]:
def gen_df_digital(df_super: pd.DataFrame, df_super_count: pd.DataFrame, df_comb: pd.DataFrame, list_df_segmento: list):

    df_super_digital = pd.DataFrame()

    for df_segmento in tqdm_notebook(list_df_segmento):

        df_super_d = pd.merge(df_super, df_segmento, how="inner", on="user_id")

        df_super_d_count = df_super_d.groupby(by=["soporte", "content_id", "programa", "periodo", "programa_referencia", "periodo_referencia", "nombre", "desc_tramo"])["user_id"].nunique().to_frame(name='usuarios').reset_index()

        df_superpos_d_1 = pd.merge(df_comb, df_super_d_count, how="inner", on=["content_id", "periodo", "programa", "periodo_referencia", "programa_referencia"])
        del df_superpos_d_1['soporte_y']
        df_superpos_d_1['dia'] = df_superpos_d_1['periodo'].dt.date
        df_superpos_d_1 = df_superpos_d_1.rename(columns={"soporte_x": "soporte", "nombre": "tipo_tramo", "desc_tramo": "segmento"}).copy()
        df_superpos_d_1 = df_superpos_d_1[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "tipo_tramo", "segmento"]]
        df_superpos_d_1.usuarios = df_superpos_d_1.usuarios.fillna(0)

        df_superpos_d_2 = df_superpos_d_1.copy()
        df_superpos_d_2['key_periodo'] = df_superpos_d_2['periodo_referencia']
        df_superpos_d_2['key_programa'] = df_superpos_d_2['programa_referencia']
        
        df_super_count = df_super_count.rename(columns={'periodo': 'key_periodo', 'programa': 'key_programa'})
        
        df_superpos_d = pd.merge(df_superpos_d_2, df_super_count, how="left", on=["key_periodo", "periodo_referencia", "key_programa", "programa_referencia", "content_id"])
        df_superpos_d = df_superpos_d.drop(["key_periodo", "key_programa", "soporte_y"], axis=1).copy()
        df_superpos_d = df_superpos_d.rename(columns={"soporte_x": "soporte", "usuarios_x": "usuarios", "usuarios_y": "total"})
        df_superpos_d = df_superpos_d[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "total", "tipo_tramo", "segmento"]]

        df_superpos_d.total = df_superpos_d.total.fillna(0)
        df_superpos_d["usuarios"] = df_superpos_d['usuarios'].astype('int64')
        df_superpos_d["total"] = df_superpos_d['total'].astype('int64')
        df_superpos_d = df_superpos_d[df_superpos_d['segmento'].notna()]

        df_super_digital = pd.concat(objs=[df_super_digital, df_superpos_d], axis=0)

    return df_super_digital

SE CREA FUNCIÓN para obtener segmento Combinado

In [13]:
def gen_df_combinados (df_super: pd.DataFrame, df_super_count: pd.DataFrame, df_comb: pd.DataFrame, df_list_combinados: list):
    
    df_super_combinado = pd.DataFrame()
    
    for df_segmento in tqdm_notebook(df_list_combinados):
        
        df_super_comb = pd.merge(df_super, df_segmento, how="inner", on="user_id")

        df_super_comb_count = df_super_comb.groupby(by=["soporte", "content_id", "programa", "periodo", "programa_referencia", "periodo_referencia","gender", "nombre", "desc_tramo"])["user_id"].nunique().to_frame(name='usuarios').reset_index()

        df_superpos_comb_1 = pd.merge(df_comb, df_super_comb_count, how="inner", on=["content_id", "periodo", "programa", "periodo_referencia", "programa_referencia"])
        del df_superpos_comb_1['soporte_y']
        df_superpos_comb_1['dia'] = df_superpos_comb_1['periodo'].dt.date
        df_superpos_comb_1 = df_superpos_comb_1.rename(columns={"soporte_x": "soporte", "nombre": "tipo_tramo", "desc_tramo": "segmento"}).copy()
        df_superpos_comb_1['tipo_tramo'] = 'Género ' + (df_superpos_comb_1['tipo_tramo'])
        df_superpos_comb_1['segmento'] = (df_superpos_comb_1['gender'] + ' ' + df_superpos_comb_1['segmento'] )
        del df_superpos_comb_1['gender']
        df_superpos_comb_1 = df_superpos_comb_1[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "tipo_tramo", "segmento"]]
        df_superpos_comb_1.usuarios = df_superpos_comb_1.usuarios.fillna(0)

        df_superpos_comb_2 = df_superpos_comb_1.copy()
        df_superpos_comb_2['key_periodo'] = df_superpos_comb_2['periodo_referencia']
        df_superpos_comb_2['key_programa'] = df_superpos_comb_2['programa_referencia']

        df_super_count = df_super_count.rename(columns={'periodo': 'key_periodo', 'programa': 'key_programa'})

        df_superpos_comb = pd.merge(df_superpos_comb_2, df_super_count, how="left", on=["key_periodo", "periodo_referencia", "key_programa", "programa_referencia", "content_id"])
        df_superpos_comb = df_superpos_comb.drop(["key_periodo", "key_programa", "soporte_y"], axis=1).copy()
        df_superpos_comb = df_superpos_comb.rename(columns={"soporte_x": "soporte", "usuarios_x": "usuarios", "usuarios_y": "total"})
        df_superpos_comb = df_superpos_comb[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios", "total", "tipo_tramo", "segmento"]]

        df_superpos_comb.total = df_superpos_comb.total.fillna(0)
        df_superpos_comb["usuarios"] = df_superpos_comb['usuarios'].astype('int64')
        df_superpos_comb["total"] = df_superpos_comb['total'].astype('int64')
        df_superpos_comb = df_superpos_comb[df_superpos_comb['segmento'].notna()]

        df_super_combinado = pd.concat(objs=[df_super_combinado, df_superpos_comb], axis=0)

    return df_super_combinado


SE OBTIENE DF_SUPERPOSICION_PROGRAMAS SIN SEGMENTAR

In [14]:
""" Se une comb con super(count) y nuevamente con super(count) para primero unir grilla con consumo 
    obteniendo cantidad de usuarios que consumieron programa de referencia y el programa indicado...
"""
df_superposicion_p_1 = pd.merge(df_comb, df_super_count, how="left", on=["content_id", "periodo", "programa", "periodo_referencia", "programa_referencia"])
del df_superposicion_p_1['soporte_y']
df_superposicion_p_1['dia'] = df_superposicion_p_1['periodo'].dt.date
df_superposicion_p_1 = df_superposicion_p_1.rename(columns={"soporte_x": "soporte"}).copy()
df_superposicion_p_1 = df_superposicion_p_1[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia", "periodo_referencia", "hora_referencia", "usuarios"]]
df_superposicion_p_1.usuarios = df_superposicion_p_1.usuarios.fillna(0)

"""Necesitamos unir periodo_referencia a periodo y a periodo_referencia a la vez
    Lo mismo con programa_referencia a programa y programa_referencia
    y unir content_id con content_id

    Debemos crear en df_superposición 2 una columna periodo aparte de la columna 
    periodo que ya contiene para hacerla calzar con periodo del df super_count
    Como superposicion 2 ya tiene una columna periodo, la podemos crear como 
    'key_periodo' y en super_count cambiamos el nombre de la columna 'periodo' 
    por 'key_periodo'
    """
df_superposicion_p_2 = df_superposicion_p_1.copy()
df_superposicion_p_2['key_periodo'] = df_superposicion_p_2['periodo_referencia']
df_superposicion_p_2['key_programa'] = df_superposicion_p_2['programa_referencia']

df_super_count = df_super_count.rename(columns={"periodo": "key_periodo", "programa": "key_programa"})

df_superposicion_p = pd.merge(df_superposicion_p_2, df_super_count, how="left", on=["key_periodo", "periodo_referencia", "key_programa", "programa_referencia", "content_id"])
df_superposicion_p = df_superposicion_p.drop(["key_periodo", "key_programa", "soporte_y"], axis=1).copy()
df_superposicion_p = df_superposicion_p.rename(columns={"soporte_x": "soporte", "usuarios_x": "usuarios", "usuarios_y": "total"})
df_superposicion_p.total = df_superposicion_p.total.fillna(0)

df_superposicion_p['segmento'] = 'no aplica'
df_superposicion_p['tipo_tramo'] = "Total"
df_superposicion_p["usuarios"] = df_superposicion_p['usuarios'].astype('int64')
df_superposicion_p["total"] = df_superposicion_p['total'].astype('int64')
df_superposicion_p = df_superposicion_p[["dia", "content_id", "soporte", "programa", "periodo", "hora", "programa_referencia",
                                        "periodo_referencia", "hora_referencia", "usuarios", "total", "tipo_tramo", "segmento"]]


PRUEBA CONCATENACIÓN DE SUPERPOSICION PROGRAMAS TOTAL 

In [15]:
df_superposicion_genero = gen_df_genero(df_super, df_super_count, df_comb, df_list_genero)

  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
df_superposicion_radial = gen_df_tramos(df_super, df_super_count, df_comb, df_list_tramo)

  0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
df_superposicion_digital = gen_df_digital(df_super, df_super_count, df_comb, df_list_digital)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
df_superposicion_combinado = gen_df_combinados(df_super, df_super_count, df_comb, df_list_combinados)

  0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
df_list_final = [df_superposicion_p, df_superposicion_genero,
                 df_superposicion_radial, df_superposicion_digital,df_superposicion_combinado ]

In [20]:
df_super_final = pd.concat(objs=df_list_final, axis=0)
df_super_final.reset_index(drop=True)

,dia,content_id,soporte,programa,periodo,hora,programa_referencia,periodo_referencia,hora_referencia,usuarios,total,tipo_tramo,segmento
0,2022-05-27,5c915724519bce27671c4d15,sonarfm.cl,radiotransmisor,2022-05-27 18:00:00,18:00:00,radiotransmisor,2022-05-27 18:00:00,18:00:00,17,17,Total,no aplica
1,2022-05-27,5c915724519bce27671c4d15,sonarfm.cl,radiotransmisor,2022-05-27 18:00:00,18:00:00,rock y guitarras,2022-05-27 10:00:00,10:00:00,9,66,Total,no aplica
2,2022-05-27,5c915724519bce27671c4d15,sonarfm.cl,radiotransmisor,2022-05-27 18:00:00,18:00:00,sonar héroes,2022-05-27 17:00:00,17:00:00,11,26,Total,no aplica
3,2022-05-27,5c915724519bce27671c4d15,sonarfm.cl,radiotransmisor,2022-05-27 18:00:00,18:00:00,bloque musical sonar,2022-05-27 13:00:00,13:00:00,8,49,Total,no aplica
4,2022-05-27,5c915724519bce27671c4d15,sonarfm.cl,radiotransmisor,2022-05-27 18:00:00,18:00:00,sonar deportivo,2022-05-27 09:00:00,09:00:00,6,58,Total,no aplica
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179329,2022-09-01,601415b58308405b0d11c82a,horizonte.cl,fina seleccion,2022-09-01 18:00:00,18:00:00,fina seleccion,2022-09-01 18:00:00,18:00:00,1,45,Género Digital,otro género 25 - 34
1179330,2022-09-01,5c8d6406f98fbf269f57c82c,playfm.cl,playpower,2022-09-01 18:00:00,18:00:00,playpower,2022-09-01 18:00:00,18:00:00,1,70,Género Digital,otro género 25 - 34
1179331,2022-09-01,5c8d6406f98fbf269f57c82c,playfm.cl,playpower,2022-09-01 18:00:00,18:00:00,playlist,2022-09-01 16:00:00,16:00:00,1,137,Género Digital,otro género 25 - 34
1179332,2022-09-01,5c8d6406f98fbf269f57c82c,playfm.cl,playlist,2022-09-01 16:00:00,16:00:00,playpower,2022-09-01 18:00:00,18:00:00,1,70,Género Digital,otro género 25 - 34


In [21]:
# tabla big query a utilizar
bq_table = "conexion-datos-rdf.consumo.superposicion_prog_segmentos"

client = bigquery.Client.from_service_account_json(_google_key)
schema = []
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
    #clustering_fields=["email", "envio"],
    time_partitioning=bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="dia",  # name of column to use for partitioning
    ))
# Make an API request.
job = client.load_table_from_dataframe(
    df_super_final, bq_table, job_config=job_config)
job.result()  # Wait for the job to complete.


LoadJob<project=conexion-datos-rdf, location=US, id=226f8aaf-1dd1-4bc3-9082-decbb3ae808b>